In [20]:
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

#import os
#import numpy as np
import pandas as pd
import base64
from pymongo import MongoClient
from bson.json_util import dumps

#### FIXED #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter

#import getpass to prompt for password at runtime
from getpass import getpass

###########################
# Data Manipulation / Model
###########################
# FIXED: change for your username and password and CRUD Python module name

username = "aacuser"
#password = getpass() #Prompt for password at run time, avoid embedding passwords in code
password = "Example123" #Optionally password can be hard coded, but this is a poor security practice
database = "AAC"
collection = "animals"
port = "52471"
shelter = AnimalShelter(username, password, port, database, collection)

#Buttons list will be used for tracking filter buttons clicked
buttons = [0, 0, 0, 0]

# class read method must support return of cursor object and accept projection json input
# Using Pandas to create a dataframe
dd = pd.DataFrame.from_records(shelter.read())

df = dd.loc[:, dd.columns != '_id'] #Remove the MongoDB unique _id column because the Dash module cannot encode it


#########################
# Dashboard Layout / View
#########################
#app = JupyterDash('Dash Datatable Only') JupyterDash was not loading in Jupyter Notebook
app = dash.Dash('Dash Datatable') #Dash module will start the webserver on localhost:8050

#FIXED Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # Grazioso Image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIXED Place the HTML image tag in the line below into the app.layout code according to your design
#FIXED Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Center(html.A( #Make the logo an html anchor that links to www.snhu.edu per customer documentation
        [
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'12%','width':'12%'})
        ], href='https://www.snhu.edu')),
    html.Center(html.B(html.H1('Lance Cain aka robot, SNHU CS-340 Dashboard'))), #Unique tag
    html.Hr(),
    html.Div(  
#FIXED Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    className='row',
            style={'display' : 'flex'},
                children= [
                    html.Button(id='submit-button-one',n_clicks=0,children='Water Rescue'),
                    html.Button(id='submit-button-two',n_clicks=0,children='Mountain Rescue'),
                    html.Button(id='submit-button-three',n_clicks=0,children='Disaster Rescue'),
                    html.Button(id='submit-button-four',n_clicks=0,children='Reset Filters')
                ]
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXED: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here
        style_cell={'textAlign' : 'center', 'minWidth' : 130}, #Set a minimum width for columns to avoid cramped table defaults
        editable=False,
        filter_action="native", #Allows filtering in the table based on matching values in each column
        sort_action="native", #Adds sorting buttons for each column, ascending or descending
        sort_mode="multi", #Allows multiple sorts with different columns
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native", #Adds pagination to the table output
        page_current= 0,
        page_size= 10, #10 Entries per page in the table
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



#FIXED Add code to filter interactive data table with MongoDB queries
#The following callback tracks button clicks for the four filter buttons
#The queries for each filter scenario uses regex to search the MongoDB database for dog breeds that match certain portions of a desireable breed name since the data set uses some shorthand like "Chesa" in lieu of "Chesapeake"
#The use of regex may result in the loss of  efficiency for specific indexes, but ensures all potential records matching for a desireable breed are included
@app.callback(Output('datatable-id','data'), #
              [Input('submit-button-one', 'n_clicks'), Input('submit-button-two', 'n_clicks'), Input('submit-button-three', 'n_clicks'), Input('submit-button-four', 'n_clicks'),
    ])
def on_click(bt1, bt2, bt3, bt4):
    if (int(bt1)==0 and int(bt2)==0 and int(bt3)==0 and int(bt4)==0): #Return unfiltered data if nothing has been clicked
        dd = pd.DataFrame.from_records(shelter.read())
        df = dd.loc[:, dd.columns != '_id']
    elif (int(bt1) > buttons[0]):
        buttons[0] = int(bt1)
        dd = pd.DataFrame.from_records(shelter.read({"animal_type" : "Dog", "breed": {"$regex": "(Chesa|Newfoundland|Labrador Retriever Mix)"}, "sex_upon_outcome":"Intact Female", "age_upon_outcome_in_weeks": {"$gte":26, "$lte":156}})) #Water rescue desireable breeds filtered with age
        df = dd.loc[:, dd.columns != '_id']
    elif (int(bt2) > buttons[1]):
        buttons[1] = int(bt2)
        dd = pd.DataFrame.from_records(shelter.read({"animal_type" : "Dog", "breed": {"$regex": "(German Shepherd|Alaskan Malamute|Old English Sheepdog|Siberian Huskey|Rottweiler)"}, "sex_upon_outcome":"Intact Male", "age_upon_outcome_in_weeks": {"$gte":26, "$lte":156}})) #Mountain rescue desireable breeds filtered with age
        df = dd.loc[:, dd.columns != '_id']
    elif (int(bt3) > buttons[2]):
        buttons[2] = int(bt3)
        dd = pd.DataFrame.from_records(shelter.read({"animal_type" : "Dog", "breed": {"$regex": "(Doberman Pinscher|German Shepherd|Golden Retriever|Bloodhound|Rottweiler)"}, "sex_upon_outcome":"Intact Male", "age_upon_outcome_in_weeks": {"$gte":20, "$lte":300}})) #Disaster rescue and individual tracking desireable breeds with age filter
        df = dd.loc[:, dd.columns != '_id']
    elif (int(bt4) > buttons[3]):
        buttons[3] = int(bt4)
        dd = pd.DataFrame.from_records(shelter.read()) #Fourth button resets the filter
        df = dd.loc[:, dd.columns != '_id']
    else:
        dd = pd.DataFrame.from_records(shelter.read()) #Should not be encountered but if nothing else matches, reset the filter/return all data
        df = dd.loc[:, dd.columns != '_id']
    return df.to_dict('records')

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    ###FIXED ####
    # add code for chart of your choice (e.g. pie chart) 
    # Created pie chart that tracks the percentage of breeds displayed within the data table
    # Unique breeds and values of the number shown in the table are sorted to ensure values are aligned and presented in a pie graph
    return [
        dcc.Graph(            
            #figure = px.pie(dff, values=dff['breed'].value_counts(), names=dff.breed.unique(), title='Percentage of Breeds Displayed')
            figure = px.pie(dff, values=dff['breed'].value_counts().sort_index(), names=sorted(dff['breed'].unique()), title='Percentage of Breeds Displayed')
            
        )    
    ]


#FIXED: Add in the code for your geolocation chart
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        #Updated example to dynamically change the map to show the location of the first animal in the table
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[0,13], dff.iloc[0,14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            #Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H2("Animal Name"),
                    html.P(dff.iloc[0,9])
                ])
            ])
        ])
    ]


app.run_server()

········
 * Serving Flask app "Dash Datatable" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Oct/2022 03:28:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2022 03:28:32] "GET /_dash-component-suites/dash_renderer/react@16.v1_3_0m1588250875.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2022 03:28:32] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_3_0m1588250875.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2022 03:28:32] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_3_0m1588250875.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2022 03:28:32] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_3_0m1588250875.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2022 03:28:32] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1_9_0m1588250878.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2022 03:28:32] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_9_0m1588250878.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2022 03:

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.6/site-packages/dash/dash.py", line 1

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.6/site-packages/dash/dash.py", line 1

127.0.0.1 - - [17/Oct/2022 03:28:33] "POST /_dash-update-component HTTP/1.1" 200 -


...

127.0.0.1 - - [17/Oct/2022 03:28:34] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2897, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 107, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 131, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1607, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1614, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'breed'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = s

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2897, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 107, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 131, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1607, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1614, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'breed'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = s

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2897, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 107, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 131, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1607, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1614, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'breed'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = s

127.0.0.1 - - [17/Oct/2022 03:28:34] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [17/Oct/2022 03:28:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2022 03:28:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2022 03:28:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2022 03:28:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2022 03:28:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2022 03:29:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2022 03:29:02] "POST /_dash-update-component HTTP/1.1" 200 -
